<a href="https://colab.research.google.com/github/annamucha3/Healthcare_Predictive_Model/blob/main/DSA601_Hospital_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color="lightblue"> Team: Anna Mucha, Navya Mehta, and Vipul Arora


<font color="lightblue"> This is a project for DSA-601.

<font color="lightblue">This project is intended to predict the length of stay and costs at hospitals in Western New York by using machine learning models.

<font color="lightblue">The dataset comes directly from the New York Department of Health. There is no PHI due to HIPAA, and no individual is identifiable. The dataset contains basic patient record level details. We have filtered the dataset down to only include Western New York. This covers the 8 counties which are Erie, Chautauqua, Wyoming, Cattaraugus, Niagara, Allegany, Genesee, and Orleans.

<font color="lightblue">To help us understand some of the data points from looking at the data dictonary provied by the Health Department with the dataset.

1. Zip Code - Only has the first 3 digits of the patients zip code. (we Might want to only look at the Hospital County rather than the 3 digit zip as it does not give us the exact location)
OOP stands for "out of state" zip codes.

2. Patient Disposition - The destination or status when the patient is leaving the facility (discharge)

3. CCSR Diagnosis Code - This is the Clinical Classification Software Refined for ICD-10CM (diagnosis codes, meaning what the patient was diagnosed with).This aggregates the more than 70,000 ICD-10CM diagnosis codes into over 520 clinically meaningful categories.  

4. CCSR Procedure Code - AHRQ (Agency for Healthcae Research and Quality) Clinical Classification Software Refined ICD-10 procedure code category (codes to identify what was done or given to the patient, such as surgeries, medical equitment, or medications.

5. APR DRG Code - All Patients Refined Diagnosis Related Groups. This is a classification system that classidies patients according to their reason for admission (why did they come to the hospital). This is a more specific classification.

6. APR MDC Code - Similar to the DRG code but is a higher level classification on the primary diagnosis.

7. APR Severity of Illness Code - All Patient Refined severity of illness code 0-4, indicating respectively, (1) minor, (2) moderate, (3) major, or (4)extreme severity of illness.

8. APR Risk of Mortality - All Patient Refined isk of mortality also called (APR ROM). These are indicated as Undetermined (0), Minor (1), Moderate (2), Major (3), Extreme (4).

9. Payment Typology 1 - Payment type (or insurance used) for payment for the occurence.

10. Payment Typology 2 - Payment type (or insurance used) for payment for the occurence.

11. Payment Typology 3 - Payment type (or insurance used) for payment for the occurence.

12. Birth Weight - Neonate birth weight in grams. Rounded to the nearest 100g.

* Might want to remove this field all together, as there are nulls and might not be extremely important for this.

13. Emergency Department Indicator - This is based on the submitted revenue codes from the hospital. If the record contained an Emergeny Department revenue code of 045X, the indicator is "Y", otherwise it will be "N".

14. Total Charges - Total charges for the discharge.

15. Total Costs - Total estimated cost for the discharge. This takes into consideration the payment typology or the insurance.

16. Length of Stay - Total number of days patient at an acute level and/0r other than acute care level. (discharge data - admission date) + 1. Length of stay greater than or equal to 120 days is written as 120+ day.


</font>

In [ ]:
#API Endpoint
#but only lets you take 1,000 rows. Instead I'll bring in as a csv to get all of the data.
#https://health.data.ny.gov/resource/yjgt-tq93.json?$query=SELECT%0A%20%20%60hospital_service_area%60%2C%0A%20%20%60hospital_county%60%2C%0A%20%20%60operating_certificate_number%60%2C%0A%20%20%60permanent_facility_id%60%2C%0A%20%20%60facility_name%60%2C%0A%20%20%60age_group%60%2C%0A%20%20%60zip_code_3_digits%60%2C%0A%20%20%60gender%60%2C%0A%20%20%60race%60%2C%0A%20%20%60ethnicity%60%2C%0A%20%20%60length_of_stay%60%2C%0A%20%20%60type_of_admission%60%2C%0A%20%20%60patient_disposition%60%2C%0A%20%20%60discharge_year%60%2C%0A%20%20%60ccsr_diagnosis_code%60%2C%0A%20%20%60ccsr_diagnosis_description%60%2C%0A%20%20%60ccsr_procedure_code%60%2C%0A%20%20%60ccsr_procedure_description%60%2C%0A%20%20%60apr_drg_code%60%2C%0A%20%20%60apr_drg_description%60%2C%0A%20%20%60apr_mdc_code%60%2C%0A%20%20%60apr_mdc_description%60%2C%0A%20%20%60apr_severity_of_illness_code%60%2C%0A%20%20%60apr_severity_of_illness%60%2C%0A%20%20%60apr_risk_of_mortality%60%2C%0A%20%20%60apr_medical_surgical%60%2C%0A%20%20%60payment_typology_1%60%2C%0A%20%20%60payment_typology_2%60%2C%0A%20%20%60payment_typology_3%60%2C%0A%20%20%60birth_weight%60%2C%0A%20%20%60emergency_department_indicator%60%2C%0A%20%20%60total_charges%60%2C%0A%20%20%60total_costs%60%0AWHERE%20caseless_one_of(%60hospital_service_area%60%2C%20%22Western%20NY%22)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

csv_path = "/content/Hospital_Inpatient_Discharges__SPARCS_De-Identified___2018_20240315.csv"

df = pd.read_csv(csv_path)
pd.set_option('display.max_columns', None)
df.head(5)

<ipython-input-22-c62a143cff06>:5: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


,Hospital Service Area,Hospital County,Operating Certificate Number,Permanent Facility Id,Facility Name,Age Group,Zip Code - 3 digits,Gender,Race,Ethnicity,Length of Stay,Type of Admission,Patient Disposition,Discharge Year,CCSR Diagnosis Code,CCSR Diagnosis Description,CCSR Procedure Code,CCSR Procedure Description,APR DRG Code,APR DRG Description,APR MDC Code,APR MDC Description,APR Severity of Illness Code,APR Severity of Illness Description,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Payment Typology 2,Payment Typology 3,Birth Weight,Emergency Department Indicator,Total Charges,Total Costs
0,Western NY,Erie,1401014,207,Buffalo General Medical Center,70 or Older,142,F,White,Not Span/Hispanic,8,Emergency,Inpatient Rehabilitation Facility,2018,INJ037,COMPLICATION OF OTHER SURGICAL OR MEDICAL CARE...,MST020,SUBCUTANEOUS TISSUE AND FASCIA EXCISION,721,"POST-OPERATIVE, POST-TRAUMATIC, OTHER DEVICE I...",18,INFECTIOUS AND PARASITIC DISEASES (SYSTEMIC OR...,4,Extreme,Extreme,Medical,Medicare,Blue Cross/Blue Shield,Self-Pay,NaN,True,55367.55,20121.21
1,Western NY,Erie,1401014,207,Buffalo General Medical Center,50 to 69,141,M,Other Race,Unknown,2,Urgent,Home w/ Home Health Services,2018,INJ006,"FRACTURE OF THE NECK OF THE FEMUR (HIP), INITI...",MST007,HIP ARTHROPLASTY,301,HIP JOINT REPLACEMENT,8,DISEASES AND DISORDERS OF THE MUSCULOSKELETAL ...,2,Moderate,Minor,Surgical,Miscellaneous/Other,Private Health Insurance,Self-Pay,NaN,False,48328.30,16943.32
2,Western NY,Erie,1401014,207,Buffalo General Medical Center,70 or Older,147,M,White,Not Span/Hispanic,1,Elective,Home or Self Care,2018,CIR023,OCCLUSION OR STENOSIS OF PRECEREBRAL OR CEREBR...,CAR006,CAROTID ENDARTERECTOMY AND STENTING,24,EXTRACRANIAL VASCULAR PROCEDURES,1,DISEASES AND DISORDERS OF THE NERVOUS SYSTEM,1,Minor,Minor,Surgical,Medicare,Self-Pay,NaN,NaN,False,17052.12,7673.72
3,Western NY,Erie,1401014,207,Buffalo General Medical Center,50 to 69,142,F,Black/African American,Not Span/Hispanic,6,Emergency,Home or Self Care,2018,DIG013,DIVERTICULOSIS AND DIVERTICULITIS,GIS002,COLONOSCOPY AND PROCTOSCOPY WITH BIOPSY,244,DIVERTICULITIS & DIVERTICULOSIS,6,DISEASES AND DISORDERS OF THE DIGESTIVE SYSTEM,3,Major,Moderate,Medical,Medicare,Self-Pay,Self-Pay,NaN,True,17287.12,9058.50
4,Western NY,Erie,1401014,207,Buffalo General Medical Center,30 to 49,147,M,White,Not Span/Hispanic,1,Emergency,Home or Self Care,2018,END012,CYSTIC FIBROSIS,NaN,NaN,254,OTHER DIGESTIVE SYSTEM DIAGNOSES,6,DISEASES AND DISORDERS OF THE DIGESTIVE SYSTEM,2,Moderate,Minor,Medical,Private Health Insurance,Self-Pay,NaN,NaN,True,6821.06,1987.18


In [ ]:
df.dtypes

Hospital Service Area                   object
Hospital County                         object
Operating Certificate Number             int64
Permanent Facility Id                    int64
Facility Name                           object
Age Group                               object
Zip Code - 3 digits                     object
Gender                                  object
Race                                    object
Ethnicity                               object
Length of Stay                          object
Type of Admission                       object
Patient Disposition                     object
Discharge Year                           int64
CCSR Diagnosis Code                     object
CCSR Diagnosis Description              object
CCSR Procedure Code                     object
CCSR Procedure Description              object
APR DRG Code                             int64
APR DRG Description                     object
APR MDC Code                             int64
APR MDC Descr

In [ ]:
df.rename(columns={'Zip Code - 3 digits': 'Short Zip Code'}, inplace=True)

In [ ]:
null_count = df.isnull().sum()
total_rows = len(df)
null_percentage = (null_count / total_rows) * 100
all_nulls = pd.DataFrame({'Null Count': null_count, 'Null Percentage': null_percentage})

all_nulls

,Null Count,Null Percentage
Hospital Service Area,0,0.000000
Hospital County,0,0.000000
Facility Name,0,0.000000
Age Group,0,0.000000
Short Zip Code,1411,0.842212
Gender,0,0.000000
Race,0,0.000000
Ethnicity,0,0.000000
Length of Stay,0,0.000000
Type of Admission,0,0.000000


In [ ]:
null_zip_code = df[df['Zip Code - 3 digits'].isnull()]
null_zip_code

In [ ]:
unique_zip = df['Short Zip Code'].unique()
unique_zip.tolist()

['142',
 '141',
 '147',
 '140',
 '143',
 'OOS',
 nan,
 '144',
 '145',
 '148',
 '146',
 '105',
 '119',
 '134',
 '136',
 '131',
 '132',
 '138',
 '117',
 '130',
 '104',
 '133',
 '149',
 '109',
 '120',
 '137',
 '139',
 '100',
 '121',
 '110',
 '108',
 '135',
 '112']

In [ ]:
test = df[['Short Zip Code', 'Hospital County']].drop_duplicates()
test

,Short Zip Code,Hospital County
0,142,Erie
1,141,Erie
2,147,Erie
9,140,Erie
25,143,Erie
47,OOS,Erie
64,147,Chautauqua
78,NaN,Erie
87,147,Wyoming
88,140,Wyoming


<font color="red">After doing some reserch on the 3 digit zip codes. One 3 digit zip code can correspond to several counties. Therefore we might want to not look at zip all together..

In [ ]:
# Dropping the 3rd payment typology since the 1st one seems to be a primary?

# Most nulls in the 3rd payment typology column.
df.drop(columns=['Payment Typology 3'], inplace=True)

In [ ]:
# wont be necessay for our modeling
df.drop(columns=['Operating Certificate Number'], inplace=True)

In [ ]:
# wont be necessay for our modeling
df.drop(columns=['Permanent Facility Id'], inplace=True)

In [ ]:
# 90% nulls in this column
df.drop(columns=['Birth Weight'], inplace=True)

In [ ]:
unique_MDC = df['APR MDC Description'].unique()
unique_MDC.tolist()

['INFECTIOUS AND PARASITIC DISEASES (SYSTEMIC OR UNSPECIFIED SITES)',
 'DISEASES AND DISORDERS OF THE MUSCULOSKELETAL SYSTEM AND CONNECTIVE TISSUE',
 'DISEASES AND DISORDERS OF THE NERVOUS SYSTEM',
 'DISEASES AND DISORDERS OF THE DIGESTIVE SYSTEM',
 'DISEASES AND DISORDERS OF THE CIRCULATORY SYSTEM',
 'DISEASES AND DISORDERS OF THE RESPIRATORY SYSTEM',
 'PREGNANCY, CHILDBIRTH AND THE PUERPERIUM',
 'NEWBORNS AND OTHER NEONATES WITH CONDITIONS ORIGINATING IN THE PERINATAL PERIOD',
 'DISEASES AND DISORDERS OF THE KIDNEY AND URINARY TRACT',
 'ENDOCRINE, NUTRITIONAL AND METABOLIC DISEASES AND DISORDERS',
 'DISEASES AND DISORDERS OF THE SKIN, SUBCUTANEOUS TISSUE AND BREAST',
 'DISEASES AND DISORDERS OF THE FEMALE REPRODUCTIVE SYSTEM',
 'HUMAN IMMUNODEFICIENCY VIRUS INFECTIONS',
 'DISEASES AND DISORDERS OF THE HEPATOBILIARY SYSTEM AND PANCREAS',
 'MENTAL DISEASES AND DISORDERS',
 'ALCOHOL/DRUG USE AND ALCOHOL/DRUG INDUCED ORGANIC MENTAL DISORDERS',
 'DISEASES AND DISORDERS OF THE EAR, NOSE, M

<font color="lightblue"> I want to shorten the description names. This will be useful when we do one hot encoding. Since this is the more general classification I will do it on MDC. DRG has many more values and wont be able to generalize as easily.

In [ ]:
mdc = {'APR MDC Description': ['INFECTIOUS AND PARASITIC DISEASES (SYSTEMIC OR UNSPECIFIED SITES)' ,
 'DISEASES AND DISORDERS OF THE MUSCULOSKELETAL SYSTEM AND CONNECTIVE TISSUE',
 'DISEASES AND DISORDERS OF THE NERVOUS SYSTEM',
 'DISEASES AND DISORDERS OF THE DIGESTIVE SYSTEM',
 'DISEASES AND DISORDERS OF THE CIRCULATORY SYSTEM',
 'DISEASES AND DISORDERS OF THE RESPIRATORY SYSTEM',
 'PREGNANCY, CHILDBIRTH AND THE PUERPERIUM',
 'NEWBORNS AND OTHER NEONATES WITH CONDITIONS ORIGINATING IN THE PERINATAL PERIOD',
 'DISEASES AND DISORDERS OF THE KIDNEY AND URINARY TRACT',
 'ENDOCRINE, NUTRITIONAL AND METABOLIC DISEASES AND DISORDERS',
 'DISEASES AND DISORDERS OF THE SKIN, SUBCUTANEOUS TISSUE AND BREAST',
 'DISEASES AND DISORDERS OF THE FEMALE REPRODUCTIVE SYSTEM',
 'HUMAN IMMUNODEFICIENCY VIRUS INFECTIONS',
 'DISEASES AND DISORDERS OF THE HEPATOBILIARY SYSTEM AND PANCREAS',
 'MENTAL DISEASES AND DISORDERS',
 'ALCOHOL/DRUG USE AND ALCOHOL/DRUG INDUCED ORGANIC MENTAL DISORDERS',
 'DISEASES AND DISORDERS OF THE EAR, NOSE, MOUTH AND THROAT',
 'MYELOPROLIFERATIVE DISEASES AND DISORDERS, AND POORLY DIFFERENTIATED NEOPLASM',
 'DISEASES AND DISORDERS OF THE BLOOD AND BLOOD FORMING ORGANS AND IMMUNOLOGICAL DISORDERS',
 'DISEASES AND DISORDERS OF THE MALE REPRODUCTIVE SYSTEM',
 'INJURIES, POISONINGS AND TOXIC EFFECTS OF DRUGS',
 'FACTORS INFLUENCING HEALTH STATUS AND OTHER CONTACTS WITH HEALTH SERVICES',
 'DISEASES AND DISORDERS OF THE EYE',
 'MULTIPLE SIGNIFICANT TRAUMA',
 'PRE MDC',
 'BURNS']
 }

In [ ]:
mdc_description_mapping = {

    'INFECTIOUS AND PARASITIC DISEASES (SYSTEMIC OR UNSPECIFIED SITES)' : 'Infectious Diseases',
    'DISEASES AND DISORDERS OF THE MUSCULOSKELETAL SYSTEM AND CONNECTIVE TISSUE' : 'Musculoskeletal System',
    'DISEASES AND DISORDERS OF THE NERVOUS SYSTEM' : 'Nervous System',
    'DISEASES AND DISORDERS OF THE DIGESTIVE SYSTEM' : 'Digestive System',
    'DISEASES AND DISORDERS OF THE CIRCULATORY SYSTEM' : 'Circulatory System',
    'DISEASES AND DISORDERS OF THE RESPIRATORY SYSTEM' : 'Respiratory System',
    'PREGNANCY, CHILDBIRTH AND THE PUERPERIUM' : 'Maternal Health',
    'NEWBORNS AND OTHER NEONATES WITH CONDITIONS ORIGINATING IN THE PERINATAL PERIOD' : 'Neonatal Disorders',
    'DISEASES AND DISORDERS OF THE KIDNEY AND URINARY TRACT' : 'Renal and Urinary Disorders',
    'ENDOCRINE, NUTRITIONAL AND METABOLIC DISEASES AND DISORDERS' : 'Metabolic Disorders',
    'DISEASES AND DISORDERS OF THE SKIN, SUBCUTANEOUS TISSUE AND BREAST' : 'Skin and Breast Disorders',
    'DISEASES AND DISORDERS OF THE FEMALE REPRODUCTIVE SYSTEM' : 'Gynecological Disorders',
    'HUMAN IMMUNODEFICIENCY VIRUS INFECTIONS' : 'Viral Infections',
    'DISEASES AND DISORDERS OF THE HEPATOBILIARY SYSTEM AND PANCREAS' : 'Hepatopancreatic Conditions',
    'MENTAL DISEASES AND DISORDERS' : 'Mental Disorders',
    'ALCOHOL/DRUG USE AND ALCOHOL/DRUG INDUCED ORGANIC MENTAL DISORDERS' : 'Substance-induced Disorders',
    'DISEASES AND DISORDERS OF THE EAR, NOSE, MOUTH AND THROAT' : 'ENT Disorders',
    'MYELOPROLIFERATIVE DISEASES AND DISORDERS, AND POORLY DIFFERENTIATED NEOPLASM' : 'Myeloproliferative Diseases ',
    'DISEASES AND DISORDERS OF THE BLOOD AND BLOOD FORMING ORGANS AND IMMUNOLOGICAL DISORDERS' : 'Hematological and Immunological Disorders',
    'DISEASES AND DISORDERS OF THE MALE REPRODUCTIVE SYSTEM' : 'Male Reproductive Disorders',
    'INJURIES, POISONINGS AND TOXIC EFFECTS OF DRUGS' : 'Injury and Drug Toxicity',
    'FACTORS INFLUENCING HEALTH STATUS AND OTHER CONTACTS WITH HEALTH SERVICES' : 'Healthcare Utilization Factors',
    'DISEASES AND DISORDERS OF THE EYE' : 'Eye Disorders',
    'MULTIPLE SIGNIFICANT TRAUMA' : 'Trauma',
    'PRE MDC' : 'Pre MDC',
    'BURNS' : 'Burns',
}

df['General_MDC'] = df['APR MDC Description'].map(mdc_description_mapping)

<font color="red"> Can remove the APR MDC Description column in the future since we created a general MDC to shorten the description.

<font color="lightblue"> Payment Typology 2	has about 27% nulls. I am going to impute the nulls with the Payment Typology 1.

In [ ]:
df['Payment Typology 2'].fillna(df['Payment Typology 1'], inplace=True)

<font color="lightblue"> There are only 8 columns (less than 1%) in the APR Risk of Mortality column. So I am going to drop these rows.

In [ ]:
df.dropna(subset=['APR Risk of Mortality'], inplace=True)